In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
#%matplotlib inline

Using TensorFlow backend.
/home/manish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/manish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/manish/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/manish/.local/lib/python3.6/site-packages/tensorflow/python

In [2]:
df= pd.read_csv("/home/manish/Documents/text classification model .csv")
df

,text,contract_name,status,category
0,Each party will keep and maintain at its princ...,contract-1570683215331,negative,0
1,The parties and their representatives and agen...,contract-1570683215331,negative,0
2,The parties and their agents or representative...,contract-1570683215331,positive,1
3,Nasdaq warrants and represents as of the date ...,contract-1570683215331,negative,0
4,"1 1Each party shall have the right, during the...",contract-1570769526433,negative,0
...,...,...,...,...
191,Upon request from TransactTools from time to t...,contract-1571629040849,positive,1
192,"From time to time, upon at least five (5) days...",contract-1571881198343,positive,1
193,At any time during regular business hours and ...,contract-1571881220520,positive,1
194,The Executive will at all times (whether durin...,contract-1571881277493,positive,1


In [4]:
X = df.text
Y = df.category



In [5]:
Y = pd.get_dummies(Y)


In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [7]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [8]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(100)(layer)
#     layer = Dense(256,name='FC1')(layer)
#     layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(2,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [9]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
out_layer (Dense)            (None, 2)                 202       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 110,602
Trainable params: 110,602
Non-trainable params: 0
_____________________________________________________

In [10]:
model.fit(sequences_matrix,Y_train,batch_size=64,epochs=10,validation_split=0.2)
# callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 132 samples, validate on 34 samples
Epoch 1/10
132/132 [==============================] - 1s 8ms/step - loss: 0.6925 - accuracy: 0.5758 - val_loss: 0.6925 - val_accuracy: 0.5294
Epoch 2/10
132/132 [==============================] - 0s 4ms/step - loss: 0.6920 - accuracy: 0.5985 - val_loss: 0.6923 - val_accuracy: 0.6176
Epoch 3/10
132/132 [==============================] - 0s 4ms/step - loss: 0.6913 - accuracy: 0.5682 - val_loss: 0.6922 - val_accuracy: 0.5882
Epoch 4/10
132/132 [==============================] - 1s 4ms/step - loss: 0.6910 - accuracy: 0.5909 - val_loss: 0.6922 - val_accuracy: 0.5882
Epoch 5/10
132/132 [==============================] - 1s 4ms/step - loss: 0.6904 - accuracy: 0.6136 - val_loss: 0.6921 - val_accuracy: 0.5882
Epoch 6/10
132/132 [==============================] - 1s 4ms/step - loss: 0.6901 - accuracy: 0.5909 - val_loss: 0.6920 - val_accuracy: 0.5882
Epoch 7/

In [11]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [12]:
#Evaluate the model on the test set.
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


30/30 [==============================] - 0s 4ms/step
Test set
  Loss: 0.695
  Accuracy: 0.467


In [13]:
df1= pd.read_csv("/home/manish/Documents/test dataset.csv")
df1

,text,status,category
0,Company must maintain records of any and all U...,negative,0
1,Company will maintain complete auditable recor...,negative,0
2,"Upon the Effective Date, each party represents...",negative,0
3,enforceable in accordance with their respectiv...,negative,0
4,Customer shall have the right to copy the Docu...,negative,0
5,In addition to the obligations under 8 1 and 8...,negative,0
6,Avici's obligation to indemnify Customer pursu...,negative,0
7,Products purchased under this Agreement may be...,negative,0
8,Each party may disclose the terms and conditio...,negative,0
9,SECTION 2 2 ATM Placement Company represents a...,negative,0


In [65]:
A= df1.text
print(A)
#B = df1.category

0     Company must maintain records of any and all U...
1     Company will maintain complete auditable recor...
2     Upon the Effective Date, each party represents...
3     enforceable in accordance with their respectiv...
4     Customer shall have the right to copy the Docu...
5     In addition to the obligations under 8 1 and 8...
6     Avici's obligation to indemnify Customer pursu...
7     Products purchased under this Agreement may be...
8     Each party may disclose the terms and conditio...
9     SECTION 2 2 ATM Placement Company represents a...
10    Higher One will make appropriate entries to th...
11    If required or warranted Company will pay for ...
12    The parties agree to cooperate with one anothe...
13    The Company, and any accountants, attorneys, f...
14    Such records and books of account shall at all...
15    The Company shall be permitted at any time, at...
16    Appropriate records and books of account of CB...
17    CBay shall, as directed by the Company, in

In [15]:
#A_train,A_test,B_train,B_test = train_test_split(A,B,test_size=0.15)

In [16]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(A)
sequences = tok.texts_to_sequences(A)
sequences_matrix1= sequence.pad_sequences(sequences,maxlen=max_len)

In [17]:
PROBABILITY = model.predict(sequences_matrix1)
print(PROBABILITY)
print(len(PROBABILITY))


[[0.4948009  0.50519913]
 [0.50000197 0.49999806]
 [0.50125337 0.49874663]
 [0.4996211  0.5003789 ]
 [0.49862584 0.5013741 ]
 [0.4954093  0.50459075]
 [0.49723274 0.50276726]
 [0.4968894  0.50311065]
 [0.50180644 0.49819362]
 [0.49772325 0.5022768 ]
 [0.50248355 0.4975164 ]
 [0.49530697 0.504693  ]
 [0.49983466 0.50016534]
 [0.50045246 0.49954757]
 [0.49392205 0.50607795]
 [0.49706033 0.50293964]
 [0.49474558 0.50525445]
 [0.4934475  0.5065525 ]
 [0.49741578 0.50258416]
 [0.49847028 0.50152975]
 [0.4953446  0.5046554 ]
 [0.49321574 0.5067843 ]
 [0.50186807 0.4981319 ]
 [0.49512628 0.5048737 ]
 [0.4952478  0.5047522 ]
 [0.49481678 0.50518316]
 [0.4967946  0.5032055 ]
 [0.4974057  0.50259435]
 [0.49084976 0.50915027]
 [0.49680993 0.5031901 ]]
30


In [81]:
PROBABILITY.shape

(30, 2)

In [83]:
w=np.argmax(PROBABILITY,axis=1)
print(w)
len(w)

[1 0 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1]


30

In [ ]:
m=[]
for i in w:
    if i==1:
        m.append('Positive')
    else:
        m.append('Negative')
        

In [94]:
Classification={'Text':A,
                 'Status':df1['status'],
                 'Category':df1['category'],
                 'New_status':m
                }

In [95]:
df2=pd.DataFrame(Classification)

In [98]:
df2.to_csv ("/home/manish/Documents/BinaryClassification.csv", index = False, header=True)

In [97]:
df2

,Text,Status,Category,New_status
0,Company must maintain records of any and all U...,negative,0,Positive
1,Company will maintain complete auditable recor...,negative,0,Negative
2,"Upon the Effective Date, each party represents...",negative,0,Negative
3,enforceable in accordance with their respectiv...,negative,0,Positive
4,Customer shall have the right to copy the Docu...,negative,0,Positive
5,In addition to the obligations under 8 1 and 8...,negative,0,Positive
6,Avici's obligation to indemnify Customer pursu...,negative,0,Positive
7,Products purchased under this Agreement may be...,negative,0,Positive
8,Each party may disclose the terms and conditio...,negative,0,Negative
9,SECTION 2 2 ATM Placement Company represents a...,negative,0,Positive
